### Importando as bibliotecas


In [7]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Base de dados

In [8]:
dados_diabetes = pd.read_csv('./dataset/diabetes.csv', sep=';')

### Verificando dados do dataset

In [9]:
dados_diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   gravidez                768 non-null    int64  
 1   glicose                 768 non-null    int64  
 2   pressao_sangue          768 non-null    int64  
 3   espessura_pele          768 non-null    int64  
 4   insulina                768 non-null    int64  
 5   IMC                     768 non-null    float64
 6   predisposicao_diabetes  768 non-null    float64
 7   idade                   768 non-null    int64  
 8   diabetes                768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


### Tratando e analisando dados

In [10]:
dados_diabetes.head()

,gravidez,glicose,pressao_sangue,espessura_pele,insulina,IMC,predisposicao_diabetes,idade,diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Contar número de registros de cada classe 

In [11]:
dados_diabetes.groupby(['diabetes'])['diabetes'].count()

diabetes
0    500
1    268
Name: diabetes, dtype: int64

### Separando conjunto de dados de features e target

In [12]:
X_dados = dados_diabetes.drop('diabetes', axis=1).values
X_dados


array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [13]:
y_dados = dados_diabetes['diabetes'].values
y_dados

array([1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,

### Criando a função para balancear os dados.

In [14]:
def balanceamento_dados(X_dados, y_dados):
    under_sample = RandomUnderSampler(random_state= 42)
    X_under, y_under = under_sample.fit_resample(X_dados, y_dados)

    tl = TomekLinks(sampling_strategy='all') # Trabalha com outliers
    X_under, y_under = tl.fit_resample(X_under, y_under)
    
    return X_under, y_under

### Aplicando técnica de balanceamento na base de dados

In [15]:
X_dados_balanceados, y_dados_balanceados = balanceamento_dados(X_dados, y_dados)

### Verificando balanceamento realizada

In [16]:
print(len(X_dados_balanceados), len(y_dados_balanceados))

456 456


In [17]:
# Outra forma de verificar o balanceamento
import collections
print(collections.Counter(y_dados_balanceados))

Counter({0: 228, 1: 228})


### Seprando base de treino e teste

In [18]:
def separa_treino_teste(X_dados_balanceados, y_dados_balanceados):
    X_train, X_test, y_train, y_test = train_test_split(X_dados_balanceados, 
                                                        y_dados_balanceados,
                                                        random_state=42,
                                                        test_size=0.2) # 80% para teste e 20% para teste
    return X_train, X_test, y_train, y_test

In [19]:
# Separando conjuntos de treino e teste
X_train, X_test, y_train, y_test = separa_treino_teste(X_dados_balanceados, y_dados_balanceados)

### Criando modelo de RandomForest

In [20]:
from sklearn.ensemble import RandomForestClassifier

# Criar o classificador Random Forest
classifier = RandomForestClassifier(random_state=42)

# Cria modelo de classificação
modelo = classifier.fit(X_train, y_train)

# Realiza predições
y_pred = modelo.predict(X_test)

### Features importantes

In [21]:
pd.DataFrame(modelo.feature_importances_, columns=['importancia'], 
             index = ['gravidez', 'glicose', 'pressao_sangue', 'espessura_pele', 'insulina',
                      'IMC', 'predisposicao_diabetes', 'idade']).sort_values('importancia', ascending=False)

,importancia
glicose,0.328321
IMC,0.156818
idade,0.140994
predisposicao_diabetes,0.104080
gravidez,0.080038
pressao_sangue,0.074558
insulina,0.059152
espessura_pele,0.056039


### Calculando a acurácia do modelo

In [23]:
acuraria = accuracy_score(y_test, y_pred)
print('Precisão: ', acuraria)

Precisão:  0.8043478260869565
